# Exploratory Data Analysis (EDA) for Optimizing Manufacturing Yield
**Project**: Optimizing Manufacturing Yield  
**Dataset**: Intelligent Manufacturing Dataset (Kaggle)  
**Objective**: Analyze sensor data to understand factors affecting yield, guiding optimization for manufacturing production settings. 
 
**Author**: Victoria Rios Vazquez (victoria.rios.vzz@gmail.com)

**Date**: April 2025  

## 1. Import Libraries

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline
sns.set_style("whitegrid")

## 2. Load Dataset

First, let's import it from kaggle

In [ ]:
# Load data
data_path = "../data/raw/manufacturing_6G_dataset.csv"  # Adjust if filename differs
df = pd.read_csv(data_path, parse_dates=["Timestamp"])

# Preview
print("Dataset Shape:", df.shape)
print("\nFirst 5 Rows:")
display(df.head())
print("\nColumns:", df.columns.tolist())

Let's fix the data type of the Machine IDs as they should be categories, not numeric

In [ ]:
df['Machine_ID'] = df['Machine_ID'].astype('category')

## 3. Basic Data Summary

Check data types, missing values, and summary stats, like summarizing omics features.

In [ ]:
# Data info
print("\nData Info:")
df.info()

# Missing values
print("\nMissing Values:")
print(df.isnull().sum())

# Summary statistics
print("\nSummary Statistics:")
display(df.describe())

**Conclusions:** It seems like there are no missing values in the dataset, therefore techniques such as imputation will not be required in downstream analysis.

## 4. Target Analysis: Yield
As we saw in the preview of the dataset, we have a few columns that could be used as our target for yield efficiency, some of these are:
-  **Quality_Control_Defect_Rate_%**, **Production_Speed_units_per_hr** or **Error_Rate_%**(continuous) as the target is continuous and directly tie to yield (lower defects = higher yield & higher production = higher yield) 
-  **Efficiency_Status** (categorical) this target could be useful for a classification problem, but if chosen could be a bit arbitrary (what counts as high, medium, and low efficiency? Not quantifiable = more difficult to improve).

Therefore, we will choose the **Quality_Control_Defect_Rate_%**, **Production_Speed_units_per_hr** or **Error_Rate_%** variable as our target for yield efficiency of this process. Let's look at its distribution and whether or not it is suitable for this problem.

In [ ]:
# Boxplot
plt.figure(figsize=(18, 5))  # Wider for 3 plots in a row

plt.subplot(1, 3, 1)
sns.boxplot(data=df, x="Efficiency_Status", y="Quality_Control_Defect_Rate_%", palette="Set2")
plt.title("Defect Rate by Efficiency Status")

plt.subplot(1, 3, 2)
sns.boxplot(data=df, x="Efficiency_Status", y="Error_Rate_%", palette="Set2")
plt.title("Error Rate by Efficiency Status")

plt.subplot(1, 3, 3)
sns.boxplot(data=df, x="Efficiency_Status", y="Production_Speed_units_per_hr", palette="Set2")
plt.title("Production Speed by Efficiency Status")

plt.tight_layout()
plt.show()


**Relationship Between Variables**

It seems like *Error_Rate_%* and *Production_Speed_units_per_hr* show a stronger relationship with *Efficiency_Status* based on the visualizations above.

To combine both speed and error into one performance indicator, we define a new variable:

`Yield_Score = Production_Speed_units_per_hr * (1 - Error_Rate_% / 100)`

**Interpretation**

(1 - Error_Rate_% / 100): Fraction of error-free units
→ e.g., if Error_Rate_% = 5%, then error-free rate = 0.95

Multiplied by Production_Speed_units_per_hr:
→ Estimates the number of defect-free units produced per hour

E.g. a Yield_Score == 300, means the process is expected to yield 380 defect-free units per hour.


In [ ]:
# Let's add the new variable to the dataset
df["Yield_Score"] = df["Production_Speed_units_per_hr"] * (1 - df["Error_Rate_%"] / 100)

# Let's plot it agains the other variables
plt.figure(figsize=(24, 5))  # Wider for 3 plots in a row

plt.subplot(1, 4, 1)
sns.boxplot(data=df, x="Efficiency_Status", y="Quality_Control_Defect_Rate_%", palette="Set2")
plt.title("Defect Rate by Efficiency Status")

plt.subplot(1, 4, 2)
sns.boxplot(data=df, x="Efficiency_Status", y="Error_Rate_%", palette="Set2")
plt.title("Error Rate by Efficiency Status")

plt.subplot(1, 4, 3)
sns.boxplot(data=df, x="Efficiency_Status", y="Production_Speed_units_per_hr", palette="Set2")
plt.title("Production Speed by Efficiency Status")

plt.subplot(1, 4, 4)
sns.boxplot(data=df, x="Efficiency_Status", y="Yield_Score", palette="Set2")
plt.title("Yield Score by Efficiency Status")

plt.tight_layout()
plt.show()


## 5. Feature Analysis
Let's explore the sensor features (e.g., temperature, pressure) to identify patterns, like features-yield correlations and distributions.

### Explore Numeric Features

**Distributions**

Let's explore the distributions of the numerical features, this will help us to detect their spread, range, and potential outliers.

In [ ]:
# Select only numerical columns
num_cols = df.select_dtypes(include=np.number).columns

# Define number of columns/rows for subplots
n_cols = 3
n_rows = int(np.ceil(len(num_cols) / n_cols))

# Set figure size
plt.figure(figsize=(5 * n_cols, 4 * n_rows))

# Loop through numerical columns and plot
for i, col in enumerate(num_cols, 1):
    plt.subplot(n_rows, n_cols, i)
    sns.histplot(df[col].dropna(), kde=True, bins=30, color='skyblue')
    plt.title(col)
    plt.xlabel('')
    plt.ylabel('')

plt.tight_layout()
plt.show()

Let's also look at the boxplots

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Select numerical columns
num_cols = df.select_dtypes(include='number').columns.drop('Efficiency_Status', errors='ignore')

# Melt to long format
df_melted = df.melt(id_vars="Efficiency_Status", value_vars=num_cols,
                    var_name="Feature", value_name="Value")

# Plot
plt.figure(figsize=(15, 6))
sns.boxplot(data=df_melted, x="Feature", y="Value", hue="Efficiency_Status", palette="Set2")
plt.xticks(rotation=45)
plt.title("Boxplot of Numerical Features by Efficiency Status")
plt.tight_layout()
plt.show()


The distributions of the numerical features appear to be uniform, this is not ideal for training a model as most assume normal distribution. For uniform distributions, it is recommended to apply quantile transformation (rank-based gaussianization) to approximate a normal distribution while it is reversible with the function .inverse_transform().

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import QuantileTransformer

# Select only numerical columns
num_cols = df.select_dtypes(include=np.number).columns

# Apply QuantileTransformer to all numerical columns
qt = QuantileTransformer(output_distribution='normal', random_state=0)
df_transformed = df.copy()
df_transformed[num_cols] = qt.fit_transform(df[num_cols])

# Define number of columns/rows for subplots
n_cols = 3
n_rows = int(np.ceil(len(num_cols) / n_cols))

# Set figure size
plt.figure(figsize=(5 * n_cols, 4 * n_rows))

# Loop through numerical columns and plot
for i, col in enumerate(num_cols, 1):
    plt.subplot(n_rows, n_cols, i)
    sns.histplot(df_transformed[col].dropna(), kde=True, bins=30, color='skyblue')
    plt.title(col)
    plt.xlabel('')
    plt.ylabel('')

plt.tight_layout()
plt.show()



Now the features appear normally distributed

**Correlations**

Let's see the correlation of the numeric features including the target features (defect rate) to observe any patterns (e.g. if temperature goes up, yield goes up) and potential predictors

In [ ]:
# Select numeric columns (exclude non-numeric, adjust if needed)
plt.figure(figsize=(10, 8))
sns.heatmap(df[num_cols].corr(), annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation with Yield Score")
plt.savefig("../docs/eda_plot.png")
plt.show()

print("\nCorrelations with Yield Score:")
print(df_transformed[num_cols].corr()["Yield_Score"].sort_values())

Let's also look at the scatterplots for these features

Original Distributions:

In [ ]:
# Define number of columns/rows for subplots
n_cols = 3
n_rows = int(np.ceil(len(num_cols) / n_cols))

# Set figure size
plt.figure(figsize=(5 * n_cols, 4 * n_rows))

# Loop through numerical columns and plot
for i, col in enumerate(num_cols, 1):
    plt.subplot(n_rows, n_cols, i)
    sns.scatterplot(x=col, y="Yield_Score", hue=df["Efficiency_Status"], data=df)
    plt.title(f"{col} vs. Yield Score")
    plt.xlabel(str(col))
    plt.ylabel("Yield_Score")

plt.tight_layout()
plt.show()

Transformed Distributions:

In [ ]:
# Define number of columns/rows for subplots
n_cols = 3
n_rows = int(np.ceil(len(num_cols) / n_cols))

# Set figure size
plt.figure(figsize=(5 * n_cols, 4 * n_rows))

# Loop through numerical columns and plot
for i, col in enumerate(num_cols, 1):
    plt.subplot(n_rows, n_cols, i)
    sns.scatterplot(x=col, y="Yield_Score", hue=df_transformed["Efficiency_Status"], data=df_transformed)
    plt.title(f"{col} vs. Yield Score")
    plt.xlabel(str(col))
    plt.ylabel("Yield Score")

plt.tight_layout()
plt.show()

**Conclusions:** It seems like there are already some insights about the features' contributions to the target variable. The scatter plots confirm the need to drop (leakage, strong/weak trends due to Yield_Score’s definition):
- *Production_Speed_units_per_hr*
- *Error_Rate_%* 

We might keep more informative parameters such as *Temperature_C*, *Vibration_Hz*, *Quality_Control_Defect_Rate_%*, etc., but address weak relationships with feature engineering (e.g., *Temperature_C* * *Vibration_Hz*) and non-linear models (Random Forest, MLP).

In [ ]:
keep_numerical = ['Temperature_C', 'Vibration_Hz', 'Power_Consumption_kW',
                  'Network_Latency_ms', 'Packet_Loss_%', 'Quality_Control_Defect_Rate_%', 
                  'Predictive_Maintenance_Score', 'Yield_Score']

### Categorical Features

Now, let's observe the same but for the categorical features

In [ ]:
# Select numeric columns (exclude non-numeric and date times, adjust if needed)
categorical_cols = ['Operation_Mode', 'Efficiency_Status']

for col in categorical_cols:

    plt.figure(figsize=(8, 5))
    sns.boxplot(x=str(col), y="Yield_Score", data=df_transformed, hue="Efficiency_Status")
    plt.title(f"Yield Score by {col}")
    plt.savefig(f"../docs/mode_yield_score_{col}.png")
    plt.show()
    print(f"\nYield Score by {col}:")
    print(df.groupby(col)["Yield_Score"].describe())

### Temporal Features

Let's explore the temporal data we have

In [ ]:
# In notebooks/01_eda.ipynb
print("Timestamp Range:")
print("Start:", df["Timestamp"].min())
print("End:", df["Timestamp"].max())
print("Sample Timestamps:")
print(df["Timestamp"].head())

It seems all data comprises the first three months of 2024. Let's break down the components into hours, day of the week, date, is weekend, etc.

In [ ]:
# Extract features
df["Hour"] = df["Timestamp"].dt.hour.astype(int)
df["Day_of_Week"] = df["Timestamp"].dt.dayofweek.astype(int)  # 0=Mon, 6=Sun
df["Date"] = df["Timestamp"].dt.date
df["Is_Weekend"] = df["Day_of_Week"].isin([5, 6]).astype(int)  # 1=Weekend, 0=Weekday
print("Sample Temporal Features:")
print(df[["Timestamp", "Hour", "Day_of_Week", "Date", "Is_Weekend"]].tail(n=10))

Let's visualize the temporal trends between the hour and yield score

In [ ]:
# Hourly trend (fixed)
plt.figure(figsize=(10, 5))
# Group by Hour, average only Yield_Score
hourly_yield = df.groupby("Hour")[["Yield_Score"]].mean().reset_index()
sns.lineplot(x="Hour", y="Yield_Score", data=hourly_yield)
plt.title("Average Yield_Score by Hour")
plt.savefig("../docs/yield_score_by_hour.png")
plt.show()

Daily trend:

In [ ]:
# Daily trend
plt.figure(figsize=(12, 5))
daily_yield = df.groupby("Date")["Yield_Score"].mean().reset_index()
sns.lineplot(x="Date", y="Yield_Score", data=daily_yield)
plt.title("Average Yield_Score by Date")
plt.xticks(rotation=45)
plt.savefig("../docs/yield_score_by_date.png")
plt.show()

Day of Week:

In [ ]:
# Day of Week trend
plt.figure(figsize=(8, 5))
sns.boxplot(x="Day_of_Week", y="Yield_Score", data=df)
plt.title("Yield_Score by Day of Week")
plt.savefig("../docs/yield_score_by_dayofweek.png")
plt.show()

Yield score by shift:

In [ ]:
# Define shifts
df["Shift"] = pd.cut(df["Hour"], bins=[0, 8, 16, 24], labels=["Night", "Morning", "Evening"], right=False)
plt.figure(figsize=(8, 5))
sns.boxplot(x="Shift", y="Yield_Score", data=df)
plt.title("Yield_Score by Shift")
plt.savefig("../docs/yield_score_by_shift.png")
plt.show()
print("Yield_Score by Shift:")
print(df.groupby("Shift")["Yield_Score"].describe())

Let's plot the correlations

In [ ]:
# Correlation
temporal_cols = ["Hour", "Day_of_Week", "Is_Weekend", "Yield_Score"]
plt.figure(figsize=(8, 6))
sns.heatmap(df[temporal_cols].corr(), annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation with Yield_Score (Temporal Features)")
plt.savefig("../docs/temporal_corr_plot.png")
plt.show()

Let's prepare the temporal data for modelling, hours and days are cyclic (e.g., 23 → 0). Let's use sine/cosine to encode:

In [ ]:
# Cyclic encoding for Hour
df["Hour_sin"] = np.sin(2 * np.pi * df["Hour"] / 24)
df["Hour_cos"] = np.cos(2 * np.pi * df["Hour"] / 24)
df["Day_of_Week_sin"] = np.sin(2 * np.pi * df["Day_of_Week"] / 7)
df["Day_of_Week_cos"] = np.cos(2 * np.pi * df["Day_of_Week"] / 7)

For the categorical and temporal features, we will drop:
- Date, Hour and Day_of_Week (replaced by cyclic encoding)
- Timestamp
- Efficiency_Status (information leak as it is directly defining the target variable)

We will keep:
- Hour_sin
- Hour_cos
- Day_of_Week_sin
- Day_of_Week_cos
- Is_Weekend
- Operation_Mode
- Shift

In [ ]:
keep_categorical_and_temporal = ['Operation_Mode', 'Hour_sin', 'Hour_cos', 'Day_of_Week_sin', 
                                 'Day_of_Week_cos', 'Is_Weekend', 'Shift']

## 7. Outlier Detection
Identify outliers in yield and sensors using the IQR method. The IQR method is simple, robust, and widely used for outlier detection in regression problems. It flags values below Q1 - 1.5IQR or above Q3 + 1.5IQR.

In [ ]:
# In notebooks/01_eda.ipynb
# Outlier detection using IQR
def detect_outliers_iqr(df, columns):
    outliers = pd.DataFrame()
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        # Identify outliers
        col_outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
        outliers = pd.concat([outliers, col_outliers], axis=0).drop_duplicates()
        print(f"{col} - Outliers: {len(col_outliers)} rows, Lower Bound: {lower_bound:.2f}, Upper Bound: {upper_bound:.2f}")
    return outliers

# Select numeric columns (excluding categorical and dropped columns)
numeric_cols = ["Temperature_C", "Vibration_Hz", "Power_Consumption_kW", 
                "Network_Latency_ms", "Packet_Loss_%", "Quality_Control_Defect_Rate_%", 
                "Predictive_Maintenance_Score", "Yield_Score", 
                "Hour", "Is_Weekend", "Hour_sin", "Hour_cos", 
                "Day_of_Week_sin", "Day_of_Week_cos"]

# Detect outliers
outliers = detect_outliers_iqr(df, numeric_cols)
print("\nTotal Unique Outlier Rows:", len(outliers))
print("Outlier Sample:")
print(outliers[numeric_cols].head())

# Optional: Remove outliers (uncomment to apply)
# df_clean = df[~df.index.isin(outliers.index)]
# print("Rows after removing outliers:", len(df_clean))

There are no outliers in our dataset.

In [ ]:
df["Yield_Score"].describe()

## 8. Initial Insights
- **Yield**: Ranges 42.53–499.44, uniform distribution.
- **Sensors**: No strong correlations with the target variable, uniform distribution.
- **Issues**: No missing values, no outliers, maybe distribution is too uniform and there is not a strong 
  correlation with the target variable.
- **Next Steps**: Normalize features (probably quantile transformation), model yield with multiple models MLP.

## 9. Save Cleaned Data
Export cleaned dataset for preprocessing, like saving processed omics data.

In [ ]:
# Specify the columns to keep
keep_cols = keep_categorical_and_temporal + keep_numerical

# Let's encode the categorical variables
df_encoded = pd.get_dummies(df[keep_cols], columns=['Operation_Mode', 'Shift'], drop_first=False)

# Convert boolean columns to integers (0 and 1)
df_encoded[[
    'Operation_Mode_Active', 
    'Operation_Mode_Idle', 
    'Operation_Mode_Maintenance', 
    'Shift_Night', 
    'Shift_Morning', 
    'Shift_Evening'
]] = df_encoded[[
    'Operation_Mode_Active', 
    'Operation_Mode_Idle', 
    'Operation_Mode_Maintenance', 
    'Shift_Night', 
    'Shift_Morning', 
    'Shift_Evening'
]].astype(int)

# View the encoded dataset
df_encoded.head()
df_encoded.dtypes


Export the clean dataset

In [ ]:
df_clean = df_encoded
df_clean.to_csv("../data/clean/clean_encoded_df_target_yield.csv", index=False)
print("Saved cleaned data to ../data/clean/clean_encoded_df_target_yield.csv")